# 9강: ETF 로우 데이터 수집 및 저장

이 강의에서는 10·11강에서 분석에 사용할 **원재료(로우 데이터)**를 정의하고,
**최소한의 스키마를 가진 CSV**로 저장합니다.

## 이 강의의 목표

1. ETF 분석에 필요한 3가지 핵심 데이터셋 정의
2. KIS API를 통한 데이터 수집
3. 최소한의 검증 후 CSV로 저장

## 수집할 데이터

| 데이터 | 파일명 | 용도 |
|--------|--------|------|
| ETF 기본 정보 | `data/raw/etf_info.csv` | 상품 메타, AUM, NAV 등 |
| ETF 구성 종목 | `data/raw/holdings_{ticker}.csv` | 포트폴리오 구성 분석 |
| ETF 가격 시계열 | `data/raw/price_{ticker}.csv` | 수익률/리스크 계산 |

## ⚠️ 이 강의에서 하지 않는 것

- 금융 지표 계산/해석 ❌
- 섹터/시가총액 분포 분석 ❌
- MDD, Sharpe, Beta 등 리스크 지표 ❌

위 분석들은 모두 **10강, 11강에서 진행**합니다.

## Step 0. KIS 공통 유틸 모듈 소개

이 강의에서는 이미 준비된 `kis_utils.py` 모듈을 사용합니다.

**모듈의 역할:**
- `.env`에서 KIS API 키/시크릿/토큰을 읽기
- 토큰 발급/재발급 처리
- `call_kis_get()` 으로 KIS API 공통 호출
- `get_etf_name_from_kis()` 로 ETF 코드 → 이름 조회

이번 ETF 강의에서는 **KIS 인증/통신 구현 자체를 설명하지 않고**,
**ETF 데이터 수집·분석 로직에만 집중**합니다.

In [1]:
import pandas as pd
from datetime import date, datetime, timedelta
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple

from kis_utils import (
    DATA_DIR,
    to_float,
    ensure_kis_token,
    call_kis_get,
    get_etf_name_from_kis,
)

## Step 0-1. 저장 디렉터리 설정

수집한 로우 데이터는 `data/raw/` 디렉터리에 저장합니다.

In [2]:
# 로우 데이터 저장 디렉터리
RAW_DATA_DIR = DATA_DIR / "raw"
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"RAW_DATA_DIR: {RAW_DATA_DIR}")

RAW_DATA_DIR: /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw


---

## Step 1. ETF 기본 정보 수집 (`etf_info.csv`)

### 1-1. ETF/ETN 현재가 API (`FHPST02400000`)

**API 정보:**
- PATH: `/uapi/etfetn/v1/quotations/inquire-price`
- TR_ID: `FHPST02400000`

**수집하는 주요 필드:**

| 컬럼명 | 설명 | 후속 활용 |
|--------|------|----------|
| `stck_prpr` | 현재가 | 괴리율 계산 |
| `nav` | 순자산가치 | 괴리율 계산 |
| `etf_ntas_ttam` | 순자산총액(AUM) | 규모 분석 |
| `mbcr_name` | 운용사명 | 메타 정보 |
| `stck_lstn_date` | 상장일 | 운용 기간 |
| `etf_div_name` | ETF 구분명 | 레버리지/인버스 판별 |
| `etf_dvdn_cycl` | 배당주기 | 배당 분석 |
| `dprt` | 괴리율 | 추적 성과 |
| `trc_errt` | 추적오차 | 추적 성과 |

In [3]:
def fetch_etf_info_raw(etf_code: str) -> Dict[str, Any]:
    """
    ETF/ETN 현재가 API를 호출하여 ETF 기본 정보를 수집.

    반환: 원본 필드 그대로 담은 dict
    """
    path = "/uapi/etfetn/v1/quotations/inquire-price"
    tr_id = "FHPST02400000"

    etf_code = str(etf_code).strip().zfill(6)

    params = {
        "fid_input_iscd": etf_code,
        "fid_cond_mrkt_div_code": "J",
    }

    data = call_kis_get(path, tr_id, params)
    output = data.get("output") or {}

    if not output:
        raise RuntimeError(f"ETF 정보 조회 실패: etf_code={etf_code}")

    # 기본 정보 추출 (원본 그대로)
    row = {
        "ticker": etf_code,
        "name_kor": get_etf_name_from_kis(etf_code),
        # "name_kor": f"{output.get('mbcr_name', '')} {output.get('etf_rprs_bstp_kor_isnm', '')}".strip(),

        # 상장/규모
        "listing_date": output.get("stck_lstn_date"),
        "aum": output.get("etf_ntas_ttam"),
        "lstn_stcn": output.get("lstn_stcn"),  # 상장좌수

        # 거래/유동성
        "last_price": output.get("stck_prpr"),
        "daily_volume": output.get("acml_vol"),
        "daily_tr_value": output.get("acml_tr_pbmn"),
        "prdy_volume": output.get("prdy_vol"),

        # NAV 관련
        "nav": output.get("nav"),
        "nav_prev": output.get("prdy_last_nav"),
        "dprt": output.get("dprt"),  # 괴리율
        "trc_errt": output.get("trc_errt"),  # 추적오차

        # 구조/유형
        "mbcr_name": output.get("mbcr_name"),  # 운용사명
        "etf_div_name": output.get("etf_div_name"),  # ETF 구분명
        "etf_dvdn_cycl": output.get("etf_dvdn_cycl"),  # 배당주기
        "etf_rprs_bstp_kor_isnm": output.get("etf_rprs_bstp_kor_isnm"),  # 대표 지수명

        # 기타 참고용
        "frgn_hldn_qty_rate": output.get("frgn_hldn_qty_rate"),  # 외국인 보유율
        "lp_hldn_rate": output.get("lp_hldn_rate"),  # LP 보유율
        "stck_dryy_hgpr": output.get("stck_dryy_hgpr"),  # 연중 최고
        "stck_dryy_lwpr": output.get("stck_dryy_lwpr"),  # 연중 최저
    }

    return row

### 1-2. 여러 ETF 정보 수집 및 저장

In [4]:
import time

def collect_etf_info_batch(
    etf_codes: List[str],
    *,
    sleep_sec: float = 0.1
) -> pd.DataFrame:
    """
    여러 ETF에 대해 기본 정보를 배치로 수집.
    """
    records = []
    
    for idx, code in enumerate(etf_codes, start=1):
        code = str(code).strip().zfill(6)
        print(f"[{idx}/{len(etf_codes)}] ETF 정보 수집: {code}")
        
        try:
            row = fetch_etf_info_raw(code)
            print(f"code: {row['ticker']}, name: {row['name_kor']}")
            records.append(row)
        except Exception as e:
            print(f"  ⚠️ 수집 실패: {e}")
            continue
        
        time.sleep(sleep_sec)
    
    if not records:
        raise RuntimeError("ETF 정보 수집 결과가 없습니다.")
    
    df = pd.DataFrame(records)
    df["as_of_date"] = date.today().isoformat()
    
    return df


def save_etf_info(df: pd.DataFrame, filename: str = "etf_info.csv") -> Path:
    """
    ETF 기본 정보 DataFrame을 CSV로 저장.
    """
    path = RAW_DATA_DIR / filename
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {path}")
    return path

### 1-3. 예시: KODEX 200 등 주요 ETF 정보 수집

In [5]:
# 예시 ETF 목록
EXAMPLE_ETFS = [
    "069500",  # KODEX 200
    "102110",  # TIGER 200
    "229200",  # KODEX KOSDAQ150
    "122630",  # KODEX 레버리지
    "252670",  # KODEX 200선물인버스2X
]

# 토큰 확인
ensure_kis_token()

# ETF 정보 수집
df_etf_info = collect_etf_info_batch(EXAMPLE_ETFS)

# 저장
save_etf_info(df_etf_info)

[1/5] ETF 정보 수집: 069500
code: 069500, name: 삼성 KODEX200 증권상장지수투자신탁[주식]
[2/5] ETF 정보 수집: 102110
code: 102110, name: 미래에셋 TIGER 200 증권상장지수투자신탁(주식)
[3/5] ETF 정보 수집: 229200
code: 229200, name: 삼성 KODEX 코스닥 150 증권상장지수투자신탁(주식)
[4/5] ETF 정보 수집: 122630
code: 122630, name: 삼성KODEX레버리지증권상장지수투자신탁[주식-파생형]
[5/5] ETF 정보 수집: 252670
code: 252670, name: 삼성 KODEX 200선물인버스2X증권상장지수투자신탁(주식-파생형)
[SAVE] /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw/etf_info.csv


PosixPath('/Users/ulift/workspace/py-etf-mango/script2/claude/data/raw/etf_info.csv')

### 1-4. 데이터 검증 (최소한의 확인만)

In [6]:
print("=" * 50)
print("ETF 기본 정보 검증")
print("=" * 50)

# 1. Shape
print(f"\n1. Shape: {df_etf_info.shape}")

# 2. Columns
print(f"\n2. Columns:")
print(f"   {list(df_etf_info.columns)}")

# 3. 결측 비율
print(f"\n3. 결측 비율:")
missing = df_etf_info.isna().mean()
for col, rate in missing.items():
    if rate > 0:
        print(f"   - {col}: {rate:.1%}")

# 4. 키 중복 체크
print(f"\n4. ticker 중복: {df_etf_info['ticker'].duplicated().sum()}개")

# 5. 샘플 데이터
print(f"\n5. 샘플 데이터:")
display(df_etf_info[["ticker", "name_kor", "aum", "listing_date", "etf_div_name"]].head())

ETF 기본 정보 검증

1. Shape: (5, 22)

2. Columns:
   ['ticker', 'name_kor', 'listing_date', 'aum', 'lstn_stcn', 'last_price', 'daily_volume', 'daily_tr_value', 'prdy_volume', 'nav', 'nav_prev', 'dprt', 'trc_errt', 'mbcr_name', 'etf_div_name', 'etf_dvdn_cycl', 'etf_rprs_bstp_kor_isnm', 'frgn_hldn_qty_rate', 'lp_hldn_rate', 'stck_dryy_hgpr', 'stck_dryy_lwpr', 'as_of_date']

3. 결측 비율:
   - daily_tr_value: 100.0%

4. ticker 중복: 0개

5. 샘플 데이터:


,ticker,name_kor,aum,listing_date,etf_div_name
0,069500,삼성 KODEX200 증권상장지수투자신탁[주식],107519,20021014,수익증권형
1,102110,미래에셋 TIGER 200 증권상장지수투자신탁(주식),42748,20080403,수익증권형
2,229200,삼성 KODEX 코스닥 150 증권상장지수투자신탁(주식),12149,20151001,수익증권형
3,122630,삼성KODEX레버리지증권상장지수투자신탁[주식-파생형],31541,20100222,수익증권형
4,252670,삼성 KODEX 200선물인버스2X증권상장지수투자신탁(주식-파생형),15738,20160922,수익증권형


---

## Step 2. ETF 구성 종목 수집 (`holdings_{ticker}.csv`)

### 2-1. ETF 구성종목시세 API (`FHKST121600C0`)

**API 정보:**
- PATH: `/uapi/etfetn/v1/quotations/inquire-component-stock-price`
- TR_ID: `FHKST121600C0`

**수집하는 주요 필드:**

| 컬럼명 | 설명 | 후속 활용 |
|--------|------|----------|
| `stck_shrn_iscd` | 종목 코드 | 종목 식별 |
| `hts_kor_isnm` | 종목명 | 표시용 |
| `stck_prpr` | 현재가 | 비중 계산 |
| `etf_vltn_amt` | ETF 내 평가금액 | 비중 계산 |
| `acml_vol` | 누적 거래량 | 유동성 참고 |

In [7]:
def fetch_etf_holdings_raw(etf_code: str) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    ETF 구성종목시세 API를 호출하여 구성 종목 리스트를 수집.
    
    반환:
    - df_holdings: 구성종목 DataFrame (원본 필드 그대로)
    - etf_summary: ETF 전체 요약 dict
    """
    path = "/uapi/etfetn/v1/quotations/inquire-component-stock-price"
    tr_id = "FHKST121600C0"
    
    etf_code = str(etf_code).strip().zfill(6)
    
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": etf_code,
        "FID_COND_SCR_DIV_CODE": "11216",
    }
    
    data = call_kis_get(path, tr_id, params)
    
    # output1: ETF 전체 요약
    output1 = data.get("output1") or {}
    etf_summary = {
        "etf_code": etf_code,
        "nav": output1.get("nav"),
        "etf_ntas_ttam": output1.get("etf_ntas_ttam"),
        "etf_cnfg_issu_cnt": output1.get("etf_cnfg_issu_cnt"),
        "stck_prpr": output1.get("stck_prpr"),
    }
    
    # output2: 구성종목 리스트
    rows = data.get("output2") or []
    
    if not rows:
        print(f"⚠️ ETF 구성종목이 비어 있습니다: {etf_code}")
        return pd.DataFrame(), etf_summary
    
    df = pd.DataFrame(rows).copy()
    
    # 기본 컬럼 정리
    df["as_of_date"] = date.today().isoformat()
    df["etf_code"] = etf_code
    df["stck_shrn_iscd"] = df["stck_shrn_iscd"].astype(str).str.strip().str.zfill(6)
    df["hts_kor_isnm"] = df["hts_kor_isnm"].astype(str).str.strip()
    
    # 비중 계산 (원본값 기반)
    df["etf_vltn_amt_float"] = df["etf_vltn_amt"].apply(to_float)
    total_vltn = df["etf_vltn_amt_float"].fillna(0).sum()
    if total_vltn > 0:
        df["weight"] = df["etf_vltn_amt_float"] / total_vltn
    else:
        df["weight"] = None
    
    # 컬럼 순서 정리
    preferred_cols = [
        "as_of_date", "etf_code", "stck_shrn_iscd", "hts_kor_isnm",
        "stck_prpr", "etf_vltn_amt", "etf_vltn_amt_float", "weight",
        "acml_vol", "acml_tr_pbmn", "etf_cnfg_issu_rlim",
    ]
    other_cols = [c for c in df.columns if c not in preferred_cols]
    df = df[[c for c in preferred_cols if c in df.columns] + other_cols]
    
    return df, etf_summary


def save_etf_holdings(df: pd.DataFrame, etf_code: str) -> Path:
    """
    ETF 구성종목 DataFrame을 CSV로 저장.
    """
    etf_code = str(etf_code).strip().zfill(6)
    filename = f"holdings_{etf_code}.csv"
    path = RAW_DATA_DIR / filename
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {path}")
    return path

### 2-2. 예시: KODEX 200 구성종목 수집

In [8]:
# KODEX 200 구성종목 수집
example_etf = "069500"

df_holdings, etf_summary = fetch_etf_holdings_raw(example_etf)

# 저장
if not df_holdings.empty:
    save_etf_holdings(df_holdings, example_etf)

# ETF 요약 정보
print(f"\nETF Summary: {etf_summary}")

[SAVE] /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw/holdings_069500.csv

ETF Summary: {'etf_code': '069500', 'nav': '55619.08', 'etf_ntas_ttam': '107519', 'etf_cnfg_issu_cnt': '201', 'stck_prpr': '55575'}


### 2-3. 데이터 검증 (최소한의 확인만)

In [9]:
print("=" * 50)
print("ETF 구성종목 검증")
print("=" * 50)

# 1. Shape
print(f"\n1. Shape: {df_holdings.shape}")

# 2. Columns
print(f"\n2. Columns:")
print(f"   {list(df_holdings.columns)}")

# 3. 결측 비율 (주요 컬럼만)
print(f"\n3. 결측 비율 (주요 컬럼):")
key_cols = ["stck_shrn_iscd", "hts_kor_isnm", "stck_prpr", "weight"]
for col in key_cols:
    if col in df_holdings.columns:
        rate = df_holdings[col].isna().mean()
        print(f"   - {col}: {rate:.1%}")

# 4. 키 중복 체크
key_cols_check = ["etf_code", "stck_shrn_iscd", "as_of_date"]
dup_count = df_holdings.duplicated(subset=key_cols_check).sum()
print(f"\n4. 키 중복 (etf_code, stck_shrn_iscd, as_of_date): {dup_count}개")

# 5. 비중 합계 확인
weight_sum = df_holdings["weight"].sum()
print(f"\n5. 비중 합계: {weight_sum:.4f}")

# 6. 상위 5개 종목
print(f"\n6. 상위 5개 종목:")
display(df_holdings.nlargest(5, "weight")[["stck_shrn_iscd", "hts_kor_isnm", "weight"]])

ETF 구성종목 검증

1. Shape: (100, 19)

2. Columns:
   ['as_of_date', 'etf_code', 'stck_shrn_iscd', 'hts_kor_isnm', 'stck_prpr', 'etf_vltn_amt', 'etf_vltn_amt_float', 'weight', 'acml_vol', 'acml_tr_pbmn', 'etf_cnfg_issu_rlim', 'prdy_vrss', 'prdy_vrss_sign', 'prdy_ctrt', 'tday_rsfl_rate', 'prdy_vrss_vol', 'tr_pbmn_tnrt', 'hts_avls', 'etf_cnfg_issu_avls']

3. 결측 비율 (주요 컬럼):
   - stck_shrn_iscd: 0.0%
   - hts_kor_isnm: 0.0%
   - stck_prpr: 0.0%
   - weight: 0.0%

4. 키 중복 (etf_code, stck_shrn_iscd, as_of_date): 0개

5. 비중 합계: 1.0000

6. 상위 5개 종목:


,stck_shrn_iscd,hts_kor_isnm,weight
0,005930,삼성전자,0.270394
1,000660,SK하이닉스,0.170955
2,105560,KB금융,0.023266
3,005380,현대차,0.020370
5,034020,두산에너빌리티,0.018865


---

## Step 3. ETF 가격 시계열 수집 (`price_{ticker}.csv`)

### 3-1. 국내주식 기간별시세(일봉) API (`FHKST03010100`)

**API 정보:**
- PATH: `/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice`
- TR_ID: `FHKST03010100`

**수집하는 주요 필드:**

| 컬럼명 | 설명 | 후속 활용 |
|--------|------|----------|
| `stck_bsop_date` | 영업일자 | 시간축 |
| `stck_clpr` | 종가 | 수익률 계산 |
| `stck_oprc` | 시가 | 변동성 참고 |
| `stck_hgpr` | 고가 | 변동성 참고 |
| `stck_lwpr` | 저가 | 변동성 참고 |
| `acml_vol` | 누적 거래량 | 유동성 지표 |

**제한사항:** 1회 호출당 최대 100개 레코드만 반환

In [10]:
def fetch_daily_price_once(
    code: str,
    start_date: str,
    end_date: str,
    *,
    period_div: str = "D",
    org_adj_prc: str = "0",  # 0: 수정주가
) -> pd.DataFrame:
    """
    국내주식 기간별시세 API를 '한 번' 호출해서 최대 100개 일봉을 반환.
    
    Args:
        code: 종목코드 (6자리)
        start_date: 시작일 (YYYYMMDD)
        end_date: 종료일 (YYYYMMDD)
    """
    path = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    tr_id = "FHKST03010100"
    
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",
        "FID_INPUT_ISCD": str(code).zfill(6),
        "FID_INPUT_DATE_1": start_date,
        "FID_INPUT_DATE_2": end_date,
        "FID_PERIOD_DIV_CODE": period_div,
        "FID_ORG_ADJ_PRC": org_adj_prc,
    }
    
    data = call_kis_get(path, tr_id, params)
    rows = data.get("output2") or []
    
    if not rows:
        print(f"⚠️ 일봉 데이터 없음: {code}, {start_date}~{end_date}")
        return pd.DataFrame()
    
    df = pd.DataFrame(rows)
    
    # 기본 컬럼 정리
    df["code"] = str(code).zfill(6)
    df["date"] = pd.to_datetime(df["stck_bsop_date"], format="%Y%m%d")
    
    # 숫자 변환
    for col in ["stck_clpr", "stck_oprc", "stck_hgpr", "stck_lwpr", "acml_vol", "acml_tr_pbmn"]:
        if col in df.columns:
            df[col + "_float"] = df[col].apply(to_float)
    
    df = df.sort_values("date").reset_index(drop=True)
    
    return df

### 3-2. 긴 기간 데이터 수집 (청크 단위 반복 호출)

In [11]:
def fetch_daily_price_range(
    code: str,
    start_date: str,
    end_date: str,
    *,
    chunk_size: int = 90,
) -> pd.DataFrame:
    """
    긴 기간의 일봉 데이터를 청크 단위로 반복 호출하여 수집.
    
    Args:
        code: 종목코드 (6자리)
        start_date: 시작일 (YYYYMMDD)
        end_date: 종료일 (YYYYMMDD)
        chunk_size: 한 번에 가져올 거래일 수 (기본 90일)
    """
    dt_start = datetime.strptime(start_date, "%Y%m%d").date()
    dt_end = datetime.strptime(end_date, "%Y%m%d").date()
    
    if dt_start > dt_end:
        raise ValueError(f"start_date({start_date}) > end_date({end_date})")
    
    frames = []
    cur_start = dt_start
    
    while cur_start <= dt_end:
        cur_end = min(cur_start + timedelta(days=chunk_size - 1), dt_end)
        
        s = cur_start.strftime("%Y%m%d")
        e = cur_end.strftime("%Y%m%d")
        
        print(f"  [INFO] fetch {code}: {s} ~ {e}")
        
        df_chunk = fetch_daily_price_once(code, s, e)
        if not df_chunk.empty:
            frames.append(df_chunk)
        
        cur_start = cur_end + timedelta(days=1)
        time.sleep(0.1)  # Rate limit
    
    if not frames:
        print(f"⚠️ 일봉 데이터를 가져오지 못했습니다: {code}")
        return pd.DataFrame()
    
    df_all = pd.concat(frames, ignore_index=True)
    df_all = df_all.sort_values("date").drop_duplicates(subset=["code", "date"]).reset_index(drop=True)
    
    return df_all


def save_etf_price(df: pd.DataFrame, etf_code: str) -> Path:
    """
    ETF 가격 시계열 DataFrame을 CSV로 저장.
    """
    etf_code = str(etf_code).strip().zfill(6)
    filename = f"price_{etf_code}.csv"
    path = RAW_DATA_DIR / filename
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {path}")
    return path

### 3-3. 예시: KODEX 200 최근 1년 가격 시계열 수집

In [12]:
# KODEX 200 가격 시계열 수집 (최근 1년)
example_etf = "069500"

end_dt = date.today()
start_dt = end_dt - timedelta(days=365)

start_date = start_dt.strftime("%Y%m%d")
end_date = end_dt.strftime("%Y%m%d")

print(f"수집 기간: {start_date} ~ {end_date}")

df_price = fetch_daily_price_range(example_etf, start_date, end_date)

# 저장
if not df_price.empty:
    save_etf_price(df_price, example_etf)

수집 기간: 20241201 ~ 20251201
  [INFO] fetch 069500: 20241201 ~ 20250228
  [INFO] fetch 069500: 20250301 ~ 20250529
  [INFO] fetch 069500: 20250530 ~ 20250827
  [INFO] fetch 069500: 20250828 ~ 20251125
  [INFO] fetch 069500: 20251126 ~ 20251201
[SAVE] /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw/price_069500.csv


### 3-4. 데이터 검증 (최소한의 확인만)

In [13]:
print("=" * 50)
print("ETF 가격 시계열 검증")
print("=" * 50)

# 1. Shape
print(f"\n1. Shape: {df_price.shape}")

# 2. Columns
print(f"\n2. Columns:")
print(f"   {list(df_price.columns)}")

# 3. 기간 확인
print(f"\n3. 기간:")
print(f"   - 시작: {df_price['date'].min()}")
print(f"   - 종료: {df_price['date'].max()}")
print(f"   - 거래일 수: {len(df_price)}")

# 4. 결측 비율 (주요 컬럼만)
print(f"\n4. 결측 비율 (주요 컬럼):")
key_cols = ["date", "stck_clpr_float", "acml_vol_float"]
for col in key_cols:
    if col in df_price.columns:
        rate = df_price[col].isna().mean()
        print(f"   - {col}: {rate:.1%}")

# 5. 키 중복 체크
dup_count = df_price.duplicated(subset=["code", "date"]).sum()
print(f"\n5. 키 중복 (code, date): {dup_count}개")

# 6. 샘플 데이터
print(f"\n6. 샘플 데이터 (최근 5일):")
display(df_price.tail()[["date", "stck_clpr_float", "acml_vol_float"]])

ETF 가격 시계열 검증

1. Shape: (242, 21)

2. Columns:
   ['stck_bsop_date', 'stck_clpr', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'acml_vol', 'acml_tr_pbmn', 'flng_cls_code', 'prtt_rate', 'mod_yn', 'prdy_vrss_sign', 'prdy_vrss', 'revl_issu_reas', 'code', 'date', 'stck_clpr_float', 'stck_oprc_float', 'stck_hgpr_float', 'stck_lwpr_float', 'acml_vol_float', 'acml_tr_pbmn_float']

3. 기간:
   - 시작: 2024-12-02 00:00:00
   - 종료: 2025-12-01 00:00:00
   - 거래일 수: 242

4. 결측 비율 (주요 컬럼):
   - date: 0.0%
   - stck_clpr_float: 0.0%
   - acml_vol_float: 0.0%

5. 키 중복 (code, date): 0개

6. 샘플 데이터 (최근 5일):


,date,stck_clpr_float,acml_vol_float
237,2025-11-25,54605.0,13967143.0
238,2025-11-26,56080.0,14646050.0
239,2025-11-27,56475.0,12038706.0
240,2025-11-28,55650.0,11190217.0
241,2025-12-01,55575.0,7305455.0


---

## 정리: 수집된 로우 데이터

이 강의에서 수집한 3가지 핵심 데이터셋입니다.

In [14]:
import os

print("=" * 60)
print("9강 수집 완료: 로우 데이터 목록")
print("=" * 60)

# 저장된 파일 목록
print(f"\n저장 디렉터리: {RAW_DATA_DIR}")
print(f"\n저장된 파일:")

for f in sorted(RAW_DATA_DIR.glob("*.csv")):
    size_kb = f.stat().st_size / 1024
    print(f"  - {f.name} ({size_kb:.1f} KB)")

print("\n" + "=" * 60)
print("10강에서는 이 데이터를 읽어서 기본·포트폴리오 지표를 계산합니다.")
print("11강에서는 가격 시계열로 수익률·리스크 지표를 계산합니다.")
print("=" * 60)

9강 수집 완료: 로우 데이터 목록

저장 디렉터리: /Users/ulift/workspace/py-etf-mango/script2/claude/data/raw

저장된 파일:
  - etf_info.csv (1.4 KB)
  - holdings_069500.csv (15.3 KB)
  - price_069500.csv (35.1 KB)

10강에서는 이 데이터를 읽어서 기본·포트폴리오 지표를 계산합니다.
11강에서는 가격 시계열로 수익률·리스크 지표를 계산합니다.


---

## 📌 참고: 벤치마크 지수 시계열

벤치마크 지수(KOSPI, KOSPI200 등) 시계열은 **11강**에서 "위험·상대지표" 설명할 때 함께 수집합니다.

API 정보:
- PATH: `/uapi/domestic-stock/v1/quotations/inquire-daily-indexchartprice`
- TR_ID: `FHKUP03500100`
- `FID_COND_MRKT_DIV_CODE`: "U" (업종)

지수 코드 예시:
- `0001`: KOSPI
- `2001`: KOSPI 200
- `2203`: KOSDAQ 150